In [1]:
import time

import torch
import transformers
from transformers import AutoTokenizer

model = "meta-llama/Llama-2-13b-chat-hf"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

start_time = time.time()
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    temperature=0.1,
    torch_dtype='auto',
    # device_map="auto",
    device=device,
)
end_time = time.time()
print(f"--- {end_time - start_time} seconds ---")

q = "What professorship did Father Josh Carrier hold at Notre Dame?"
question =  f'Question: {q}\nAnswer:'

start_time = time.time()
sequences = pipeline(
    question,
    do_sample=True,
    top_k=50,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=50,
)
end_time = time.time()
print(f"--- {end_time - start_time} seconds ---")

for seq in sequences:
    print(seq['generated_text'])
    output = seq['generated_text']

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

--- 222.06616067886353 seconds ---
--- 1.8400397300720215 seconds ---
Question: What professorship did Father Josh Carrier hold at Notre Dame?
Answer: Father Josh Carrier held the professorship of New Testament Studies at the University of Notre Dame.

Question: What is the name of Father Josh


In [2]:
### compute bleu score

ref = """Yes, definitely! Based on your interest in "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Sopranos" - This HBO series is a crime drama that explores the life of a New Jersey mob boss, Tony Soprano, as he navigates the criminal underworld and deals with personal and family issues.

2. "The Wire" - This HBO series is a gritty and realistic portrayal of the drug trade in Baltimore, exploring the impact of drug addiction on individuals and communities. The show features a talented ensemble cast and is known for its complex characters and nuanced storytelling.

3. "Narcos" - This Netflix series tells"""

ref = "Professor of Chemistry and Physics"

ref = ref.replace(question+" ", "").replace("\n", " ")
output = output.replace(question+" ", "").replace("\n", " ")

print(ref, "\n==========================\n", output)

# Answer: Of course! Based on your interest in "Breaking Bad" and "Band of Brothers," here are some other TV shows you might enjoy:
# 1. "The Sopranos" - This HBO series is a crime drama that explores the life of a New Jersey mob boss, Tony Soprano, as he navigates the criminal underworld and deals with personal and family issues.
# 2. "Mad Men" - Set in the 1960s, this AMC series follows the lives of advertising executives on Madison Avenue, exploring themes of identity, power, and the changing


import evaluate

predictions = [output]
references = [[ref],]
sacrebleu = evaluate.load("sacrebleu")
results = sacrebleu.compute(predictions=predictions, references=references)
print(results)
print(list(results.keys()))
print(round(results["score"], 4))

def EvaluateBotGoldStandard(bot_reply, gol_standard):
    bot_reply = bot_reply.replace(question+" ", "").replace("\n", " ")
    gol_standard = gol_standard.replace(question+" ", "").replace("\n", " ")
    predictions = [bot_reply]
    references = [[gol_standard],]
    sacrebleu = evaluate.load("sacrebleu")
    results = sacrebleu.compute(predictions=predictions, references=references)

    return results, results["score"]


Professor of Chemistry and Physics 
 Father Josh Carrier held the professorship of New Testament Studies at the University of Notre Dame.  Question: What is the name of Father Josh
{'score': 1.4445809981770859, 'counts': [1, 0, 0, 0], 'totals': [26, 25, 24, 23], 'precisions': [3.8461538461538463, 2.0, 1.0416666666666667, 0.5434782608695652], 'bp': 1.0, 'sys_len': 26, 'ref_len': 5}
['score', 'counts', 'totals', 'precisions', 'bp', 'sys_len', 'ref_len']
1.4446


In [3]:
def EvaluateBotQAAnswer(bot_reply, qa_answers):
    for answer in qa_answers:
        match = True
        for word in answer.split():
            word = word.lower()
            if word in bot_reply.lower():
                continue
            else:
                match = False
                break

        if match:
            break

    return int(match)

start_time = time.time()
bot_reply = "Father Josh Carrier held the professorship of New Testament Studies at the University of Notre Dame."
qa_answers = ["fatther josh carrier", "New Testament Studies at the University of Notre Dame"]
end_time = time.time()
print(f"--- {end_time - start_time} seconds ---")

print(EvaluateBotQAAnswer(bot_reply, qa_answers))

--- 8.106231689453125e-05 seconds ---
1
